In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.datasets import mnist

import numpy as np
import matplotlib.pyplot as plt

import os
import io

import datetime

%load_ext tensorboard

2022-11-07 10:30:12.140112: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 10:30:12.859645: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 10:30:13.970574: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/simon/anaconda3/envs/tf/lib/
2022-11-07 10:30:13.970703: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.

In [3]:
IMAGE_HEIGHT = 64
IMAGE_WIDTH = 64
BATCH_SIZE = 256

In [4]:
AUTOTUNE = tf.data.AUTOTUNE
dataset = tf.keras.utils.image_dataset_from_directory("dataset/preprocessed",
  image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
  batch_size=BATCH_SIZE)
normalization_layer = tf.keras.layers.Rescaling(1./127.5, offset=-1)
dataset = dataset.map(lambda x, y: (normalization_layer(x), y))

Found 214692 files belonging to 2 classes.


2022-11-07 10:30:20.754268: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 10:30:20.847481: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 10:30:20.847590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 10:30:20.851621: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [8]:
def make_generator(latent_vector_shape, dense_shape):
    latent_input = layers.Input(shape=latent_vector_shape)
    gen = layers.Dense(4*4*dense_shape)(latent_input)
    gen = layers.ReLU()(gen)
    gen = layers.Reshape((4,4,dense_shape))(gen)
    gen = layers.Dropout(0.2)(gen)
    
    gen = layers.Conv2DTranspose(dense_shape, (2, 2), 2, use_bias=False)(gen)
    gen = layers.LeakyReLU()(gen)
    gen = layers.BatchNormalization()(gen)
    gen = layers.Dropout(0.25)(gen)
    
    gen = layers.Conv2DTranspose(dense_shape/2, (2, 2), 2, use_bias=False)(gen)
    gen = layers.LeakyReLU()(gen)
    gen = layers.BatchNormalization()(gen)
    gen = layers.Dropout(0.25)(gen)
    
    gen = layers.Conv2DTranspose(dense_shape/4, (2, 2), 2, use_bias=False)(gen)
    gen = layers.LeakyReLU()(gen)
    gen = layers.BatchNormalization()(gen)
    gen = layers.Dropout(0.25)(gen)
    
    gen = layers.Conv2DTranspose(dense_shape/8, (2, 2), 2, use_bias=False)(gen)
    gen = layers.LeakyReLU()(gen)
    gen = layers.BatchNormalization()(gen)
    
    out = layers.Conv2D(3, (1, 1), strides=(1,1), activation='tanh')(gen)
    
    model: keras.Model = keras.Model(latent_input, out)
    print(model.output_shape)
    assert model.output_shape == (None, 64, 64, 3)
    return model

generator = make_generator(128, 256)
generator.summary()

(None, 64, 64, 3)
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128)]             0         
                                                                 
 dense_3 (Dense)             (None, 4096)              528384    
                                                                 
 re_lu_3 (ReLU)              (None, 4096)              0         
                                                                 
 reshape_3 (Reshape)         (None, 4, 4, 256)         0         
                                                                 
 dropout_12 (Dropout)        (None, 4, 4, 256)         0         
                                                                 
 conv2d_transpose_12 (Conv2D  (None, 8, 8, 256)        262144    
 Transpose)                                                      
                                         

In [10]:
def make_discriminator(input_shape):
    image_input = layers.Input(input_shape)
    
    disc = layers.Conv2D(8, (3, 3))(image_input)
    disc = layers.AveragePooling2D()(disc)
    disc = layers.LeakyReLU(alpha=0.02)(disc)
    disc = layers.BatchNormalization()(disc)
    
    
    disc = layers.Conv2D(16, (3, 3))(disc)
    disc = layers.AveragePooling2D()(disc)
    disc = layers.LeakyReLU(alpha=0.02)(disc)
    disc = layers.Dropout(0.3)(disc)
    
    disc = layers.Conv2D(32, (3, 3))(disc)
    disc = layers.AveragePooling2D()(disc)
    disc = layers.LeakyReLU(alpha=0.02)(disc)
    
    disc = layers.Flatten()(disc)
    disc = layers.Dropout(0.3)(disc)
    disc = layers.Dense(32)(disc)
    out = layers.Dense(1)(disc)
    
    model = keras.Model(image_input, out)
    
    return model

discriminator = make_discriminator((64, 64, 3))
discriminator.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d_7 (Conv2D)           (None, 62, 62, 8)         224       
                                                                 
 average_pooling2d_3 (Averag  (None, 31, 31, 8)        0         
 ePooling2D)                                                     
                                                                 
 batch_normalization_17 (Bat  (None, 31, 31, 8)        32        
 chNormalization)                                                
                                                                 
 leaky_re_lu_19 (LeakyReLU)  (None, 31, 31, 8)         0         
                                                                 
 conv2d_8 (Conv2D)           (None, 29, 29, 16)        1168

In [11]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim=128, disc_extra_steps=3):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_steps = disc_extra_steps
        
    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.g_loss_fn = g_loss_fn

    def train_step(self, data):
        images, _ = data
        batch_size = tf.shape(images)[0]
        
        for i in range(self.d_steps):
            latent_vector = tf.random.normal(shape=(batch_size, self.latent_dim))
            with tf.GradientTape() as gt:
                generated_images = self.generator(latent_vector, training=True)
                prediction_fake = self.discriminator(generated_images, training=True)
                
                flipped_images = tf.image.random_flip_left_right(images)
                prediction_real = self.discriminator(flipped_images, training=True)
                
                d_loss = self.d_loss_fn(prediction_real, prediction_fake)
            d_gradients = gt.gradient(d_loss, self.discriminator.trainable_variables)
            self.d_optimizer.apply_gradients(zip(d_gradients, self.discriminator.trainable_variables))
            
            latent_vector = tf.random.normal(shape=(batch_size, self.latent_dim))
            with tf.GradientTape() as gt:
                generated_images = self.generator(latent_vector, training=True)
                prediction_fake = self.discriminator(generated_images, training=True)
                g_loss = self.g_loss_fn(prediction_fake)
                
            g_gradients = gt.gradient(g_loss, self.generator.trainable_variables)
            self.g_optimizer.apply_gradients(zip(g_gradients, self.generator.trainable_variables))
            
        return {"d_loss": d_loss, "g_loss": g_loss}

In [12]:
cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [13]:
checkpoint_dir = 'celebGAN/training_checkpoints'
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                discriminator_optimizer=discriminator_optimizer,
                                generator=generator,
                                discriminator=discriminator)
manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=5)

In [14]:
log_dir = "celebGAN/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
img_log_dir = "celebGAN/logs/images/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

file_writer = tf.summary.create_file_writer(img_log_dir)

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=16, latent_dim=100, start_epoch=0, seed=None):
        self.num_img = num_img
        if not seed == None:
            self.seed = seed
        else:
            self.seed = tf.random.normal(shape=(num_img, latent_dim))
        self.start_epoch = start_epoch
    
    def on_epoch_end(self, epoch, logs=None):
        generated_images = self.model.generator(self.seed, training=False)
        generated_images = (generated_images * 127.5) + 127.5
        generated_images = generated_images.numpy()
        
        fig = plt.figure(figsize=(4, 4))
    
        for i in range(generated_images.shape[0]):
            plt.subplot(4, 4, i+1)
            plt.imshow(generated_images[i, :, :, :].astype("int32"))
            plt.axis('off')

        plt.savefig(os.path.join("celebGAN/", "images/",'image_at_epoch_{:04d}.png'.format(self.start_epoch+epoch)))
        with file_writer.as_default():
            tf.summary.image("Output", plot_to_image(fig), step=epoch)
            
class GANSaver(keras.callbacks.Callback):
    def __init__(self, manager, num_epochs=15):
        self.num_epochs = num_epochs
        self.manager = manager
    
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.num_epochs == 0:
            self.manager.save()


ckp = GANSaver(manager, 2)

In [15]:
gan = GAN(discriminator, generator, latent_dim=128, disc_extra_steps=1)
gan.compile(discriminator_optimizer, generator_optimizer, discriminator_loss, generator_loss)

In [ ]:
if manager.latest_checkpoint:
    checkpoint.restore(manager.latest_checkpoint)
    latest_epoch = int(manager.latest_checkpoint.split('-')[1])
    last_epoch = latest_epoch * 2
    print ('Latest checkpoint of epoch {} restored!!'.format(last_epoch))
else:
    last_epoch = 0
    print ('No latest checkpoint found!')
ick = GANMonitor(num_img=16, latent_dim=128, start_epoch=last_epoch)

%tensorboard --logdir celebGAN2/logs
gan.fit(dataset, epochs=100, batch_size=256, callbacks=[ick, ckp, tensorboard_callback])

No latest checkpoint found!


Epoch 1/100
839/839 [==============================] - 67s 80ms/step - d_loss: 0.9380 - g_loss: 1.4768
Epoch 2/100
839/839 [==============================] - 64s 76ms/step - d_loss: 1.2716 - g_loss: 0.8865
Epoch 3/100
839/839 [==============================] - 64s 76ms/step - d_loss: 1.2865 - g_loss: 0.9095
Epoch 4/100
839/839 [==============================] - 62s 74ms/step - d_loss: 1.3103 - g_loss: 0.8172
Epoch 5/100
839/839 [==============================] - 62s 74ms/step - d_loss: 1.3586 - g_loss: 0.7354
Epoch 6/100
839/839 [==============================] - 62s 74ms/step - d_loss: 1.3258 - g_loss: 0.7970
Epoch 7/100
839/839 [==============================] - 62s 74ms/step - d_loss: 1.2891 - g_loss: 0.8873
Epoch 8/100
839/839 [==============================] - 63s 74ms/step - d_loss: 1.2845 - g_loss: 0.8451
Epoch 9/100
839/839 [==============================] - 62s 74ms/step - d_loss: 1.3475 - g_loss: 0.7999
Epoch 10/100
839/839 [==============================] - 62s 74ms/step - d

In [28]:
manager.save()
seed = ick.seed

In [ ]:
images = 25
predictions = np.empty([100,64,64,3])
for i in range(4):
    seed = tf.random.normal([images, 128])
    label_seed = np.random.randint(0,2, images)
    pred = generator(seed, training=False).numpy()
    predictions[25*i:25*(i+1), :, :, :] = pred
    
print(predictions.shape)
figsize = 10
fig = plt.figure(figsize=(figsize, figsize))  
for i in range(predictions.shape[0]):
    plt.subplot(figsize, figsize, i+1)
    plt.imshow((predictions[i, :, :, :]*127.5+127.5).astype("int32"))
    plt.axis('off')
plt.show()

In [ ]:
tf.keras.utils.plot_model(generator, "celebGAN/Generator.png", show_shapes=True)
tf.keras.utils.plot_model(discriminator, "celebGAN/Discriminator.png", show_shapes=True)

In [30]:
generator.save("celebGAN/generator_model/generator.h5")

In [31]:
generator.save_weights("celebGAN/generator_model/weights.h5")